# 자연어 처리(NLP) : IMDb 영화 리뷰 감성 분석

In [ ]:
# 라이브러리 설정
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import random

# 랜덤 시드 고정
SEED=12
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

* 순서가 있는 시쿼스 데이터 분석에 사용되는 RNN(Recurrent Neural Network) 계열의 순환 신경망

In [ ]:
# IMDb 영화 리뷰 데이터 셋

from tensorflow.keras import datasets
imdb=datasets.imdb
(x_train,y_train),(x_test,y_test)=imdb.load_data(num_words=10000,index_from=3)


In [ ]:
print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)

In [ ]:
# 첫 번째 리뷰 - 벡터
print(x_train[0])

* 각 숫자는 실제로는 단어를 나타낸다. 자연어를 딥러닝 모델에 입력하기 위해 각 단어를 숫자와 1:1 매핑하여 숫자 인덱스로 인코딩 변환했기 때문이다.

In [ ]:
# 첫 번째 리뷰 - 벡터 길이(원소 개수)
len(x_train[0])

* 218개의 단어로 이루어진 리뷰

In [ ]:
word_index=imdb.get_word_index()
word_index

In [ ]:
# 숫자 벡터를 텍스트로 변환 ( 원래 문장으로 복원할 수 있다. )
def decode_review_vector(review_vector):
  index_to_word={value:key for key, value in word_index.items()}
  decode_review=' '.join([index_to_word.get(idx-3,'?') for idx in review_vector])
  return decode_review

# 첫 번째 리뷰 디코딩
decode_review_vector(x_train[0])


In [ ]:
# 첫 번째 리뷰의 정답 레이블
y_train[0]

In [ ]:
# 각 리뷰의 단어 개수 분포

review_length=[len(review) for review in x_train]
sns.displot(review_length);

# 제로 패딩

1.   리뷰 데이터의 길이가 모두 다르다.
2.   딥러닝 모델에 입력하기 위해 입력 데이터의 크기를 동일하게 조정해야한다.

> 제로 패딩(zero padding) : 입력의 최대 길이를 설정하고, 이보다 길이가 긴 경우에는 중간에 잘라서 길이를 맞추고, 짧은 경우에는 부족한 만큼 숫자 0 으로 채운다.





In [ ]:
# Padding
from tensorflow.keras.preprocessing import sequence
x_train_pad=sequence.pad_sequences(x_train,maxlen=250)
x_test_pad=sequence.pad_sequences(x_test,maxlen=250)

print(x_train_pad[0])

# 단어 임배딩



*   IMDb 데이터를 불러올 때 num_words에 사용할 단어 개수를 10,000개로 설정했기 때문에 각 리뷰는 10,000개의 단어 사전(BoW,Back of Words)에 들어 있는 단어로 표현되어 있다. 

*   단어 사전에 등록된 10,000개 단어를 기준으로 원핫 인코딩을 적용하여 원핫 벡터로 변환하면 딥러닝 모델의 입력 데이터로 사용할 수 있다.



*   이 경우, 모델에 충분한 정보를 전달하기 어려워지고, 단어사이의 유사성 등 언어적 특징을 모델이 학습할 수 없다.
*   따라서, 단어 임베딩 활용





In [ ]:
# 모델 정의
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.layers import Embedding, SimpleRNN, LSTM,GRU

def build_model(model_type='RNN'):
  model=Sequential()

  # Embedding
  model.add(Embedding(input_dim=10000,output_dim=128))

  # RNN
  if model_type=='RNN':
    model.add(SimpleRNN(units=64,return_sequences=True))
    model.add(SimpleRNN(units=64))

  # LSTM

  elif model_type=='LSTM':
    model.add(LSTM(units=64,return_sequences=True))
    model.add(LSTM(units=64))

  # GRU

  elif model_type=='GRU':
    model.add(GRU(units=64,return_sequences=True))
    model.add(GRU(units=64))

  # Dense Classifier

  model.add(Dense(units=32,activation='relu'))
  model.add(Dropout(rate=0.5))
  model.add(Dense(units=1,activation='sigmoid'))

  # Compile
  model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])
  
  return model

# RNN

In [ ]:
rnn_model=build_model('RNN')
rnn_model.summary()

In [ ]:
rnn_history=rnn_model.fit(x_train_pad,y_train,batch_size=32,epochs=10,
                          validation_split=0.1,verbose=2)

In [ ]:
# 20 epoch까지 손실 함수와 정확도를 그래프로 나타내는 함수

def plot_metrics(history,start=1,end=20):
  flt,axes=plt.subplots(1,2,figsize=(10,5))

  # Loss 손실 함수
  axes[0].plot(range(start,end+1),history.history['loss'][start-1:end],
               label='Train')
  axes[0].plot(range(start,end+1),history.history['val_loss'][start-1:end],
               label='Validation')
  axes[0].set_title('Loss')
  axes[0].legend()

  # Accuracy : 예측 정확도
  axes[1].plot(range(start,end+1),history.history['accuracy'][start-1:end],
               label='Train')
  axes[1].plot(range(start,end+1),history.history['val_accuracy'][start-1:end],
               label='Validation')
  axes[1].set_title('Accuracy')
  axes[1].legend()
  plt.show()

# 그래프 그리기
plot_metrics(history=rnn_history,start=1,end=10)

# LSTM



1.   RNN의 단점을 보완한 모델
* RNN :최근에 학습한 단어가 모델의 최종예측에 더 큰 영향을 주게된다. RNN 모델이 단기 의존성이 높다.
* LSTM : 기존 정보 중에서 중요한 정보를 다음 단계에 전달하는 구조를 도입, 장기 기억 성능을 개선했다. 길이가 긴 시퀀스 데이터를 학습하는데 적합하다.


In [ ]:
# LSTM 모델 적용
lstm_model=build_model('LSTM')
lstm_history=lstm_model.fit(x_train_pad,y_train,batch_size=32,epochs=10,
                            validation_split=0.1,verbose=0)

plot_metrics(history=lstm_history,start=1,end=10)

# GRU 




*   LSTM의 느린 학습 속도를 개선하기 위해 제안된 알고리즘
*   LSTM은 입력 시퀀스의 길이가 길어지면, 학습해야하는 가중치 파라미터의 개수가 기하급수적으로 늘기 때문에, 학습속도가 느리다는 단점이 있다.

* GRU는 셀을 단순한 구조로 변경하여 파라미터 개수를 줄이고 학습 속도를 개선한 모델이다.



In [ ]:
# GRU 모델 적용

gru_model=build_model('GRU')
gru_model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

gru_history=gru_model.fit(x_train_pad,y_train,batch_size=32,epochs=10,
                          validation_split=0.1,verbose=0)

plot_metrics(history=gru_history,start=1,end=10)